In [27]:
import pandas as pd
import numpy as np
import subprocess
from datetime import date
import calendar
import bandit.main as bd

In [28]:
def set_cohorts(df, score_col):
    cohort_size = 3
    cohort_size_2 = cohort_size**2
    cohort_size_3 = cohort_size**3

    # sort by score
    df = df.sort_values (score_col)
    df = df.reset_index ()

    if len (df) > cohort_size:
        grp_size = len (df) / cohort_size
        df['cohort1'] = df.index // grp_size

    if len (df) > cohort_size_2:
        grp_size = len (df) / cohort_size_2
        df['cohort2'] = df.index // grp_size
        
    if len (df) > cohort_size_3:
        grp_size = len (df) / cohort_size_3
        df['cohort3'] = df.index // grp_size

    return df

In [29]:
def df_bandit_class(df, classname):
    dfgb = df.groupby([classname])
    dfgb = dfgb['Quality'].agg(['count','mean', 'var']).reset_index()
    dfgb['total'] = dfgb['count'].sum()
    dfgb['donext'] = dfgb.apply(lambda x: bd.get_tuned_ucb(x, 'mean', 'var'), axis=1).fillna(999)
    foo = dfgb.sort_values('donext', ascending=False)
    return foo

In [30]:
def df_search(df_in, cohort):
    df_class = df_bandit_class(df_in,cohort)
    print(df_class)
    nextClass = df_class.iloc[0,0]
    if df_in['Completed'].isnull().sum() > 0:
        df_filtered = df_in[df_in[cohort] == nextClass]
    else:
        df_filtered = df_in

    return df_filtered

In [31]:
currDate = date.today()
list = [[x, y] for x in range(1,15) for y in [72,76,80,84,88]]

In [32]:
df = pd.DataFrame.from_records(list).rename(index=str, columns={0: "Minutes", 1: "Rate"})
df['Date'] = currDate
df['Completed'] = np.nan
df = df[['Date','Minutes','Rate','Completed']]
sheet_info = pd.read_excel(io='PunchGoals.xlsx')
df = df.append(sheet_info).reset_index(drop=True)

In [33]:
subprocess.Popen('PunchGoals.xlsx', shell=True)

In [34]:
df['Value'] = df['Minutes'] * (df['Rate'] - 10)
df = set_cohorts(df, 'Value')
df['Score'] = np.where(df['Completed'] >= df['Minutes'], df['Value']**1.2, df['Value'] / 5)
df.loc[df['Completed'].isnull(), 'Score'] = np.NaN
df['Class'] = df['Minutes'].map(str) + '-' + df['Rate'].map(str)
df = bd.reduce(df,'Class')
df

2 0.995 2


,index,Date,Minutes,Rate,Completed,Value,cohort1,cohort2,cohort3,Score,Class
0,0,2018-10-04,1,72,NaN,62,-0.0,-0.0,-0.0,NaN,1-72
1,70,2018-10-01 00:00:00,1,72,1.0,62,0.0,0.0,0.0,141.537013,1-72
2,1,2018-10-04,1,76,NaN,66,0.0,0.0,0.0,NaN,1-76
3,2,2018-10-04,1,80,NaN,70,0.0,0.0,1.0,NaN,1-80
4,3,2018-10-04,1,84,NaN,74,0.0,0.0,1.0,NaN,1-84
5,4,2018-10-04,1,88,NaN,78,0.0,0.0,1.0,NaN,1-88
6,5,2018-10-04,2,72,NaN,124,0.0,0.0,2.0,NaN,2-72
7,6,2018-10-04,2,76,NaN,132,0.0,0.0,2.0,NaN,2-76
8,7,2018-10-04,2,80,NaN,140,0.0,1.0,3.0,NaN,2-80
9,8,2018-10-04,2,84,NaN,148,0.0,1.0,3.0,NaN,2-84


In [35]:
df_quality = bd.get_quality(df['Score'])
df = df.join(df_quality)

In [36]:
df_step1 = df_search(df,'cohort1')

   cohort1  count  mean  var  total      donext
2      2.0      0   NaN  NaN      2  999.000000
0     -0.0      1   1.0  NaN      2    1.416277
1      1.0      1   0.0  NaN      2    0.416277


In [37]:
df_step2 = df_search(df_step1,'cohort2')

   cohort2  count  mean  var  total  donext
0      6.0      0   NaN  NaN      0   999.0
1      7.0      0   NaN  NaN      0   999.0
2      8.0      0   NaN  NaN      0   999.0


In [38]:
df_step3 = df_search(df_step2,'cohort3')
df_step3

   cohort3  count  mean  var  total  donext
0     18.0      0   NaN  NaN      0   999.0
1     19.0      0   NaN  NaN      0   999.0
2     20.0      0   NaN  NaN      0   999.0


,index,Date,Minutes,Rate,Completed,Value,cohort1,cohort2,cohort3,Score,Class,Quality
48,50,2018-10-04,11,72,NaN,682,2.0,6.0,18.0,NaN,11-72,NaN
49,47,2018-10-04,10,80,NaN,700,2.0,6.0,18.0,NaN,10-80,NaN
50,44,2018-10-04,9,88,NaN,702,2.0,6.0,18.0,NaN,9-88,NaN


In [39]:
df_bandit_class(df_step3, 'Class')

,Class,count,mean,var,total,donext
0,10-80,0,NaN,NaN,0,999.0
1,11-72,0,NaN,NaN,0,999.0
2,9-88,0,NaN,NaN,0,999.0
